In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
%matplotlib inline
import seaborn as sns; sns.set_theme(color_codes=True)
pd.set_option('display.max_columns', None)
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.colors
import warnings
warnings.filterwarnings('ignore')
from scipy import stats
import math

In [2]:
labels = pd.read_csv('data/processed-labels/labels_all_oradell.csv')

In [3]:
labels

,audit_task_id,label_id,gsv_panorama_id_x,label_type,severity,correct,high_quality_user,geometry,gsv_panorama_id_y,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,lat,lng
0,7,9,8AsY4nlOJQAo8P5YYaEjzA,CurbRamp,1.0,True,True,POINT (-74.03411102294922 40.945518493652344),8AsY4nlOJQAo8P5YYaEjzA,2,309.035706,-15.723214,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945518,-74.034111
1,7,10,8AsY4nlOJQAo8P5YYaEjzA,NoSidewalk,5.0,True,True,POINT (-74.03414154052734 40.94554138183594),8AsY4nlOJQAo8P5YYaEjzA,2,303.232147,-13.580358,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945541,-74.034142
2,7,11,8AsY4nlOJQAo8P5YYaEjzA,CurbRamp,1.0,True,True,POINT (-74.0341567993164 40.9454345703125),8AsY4nlOJQAo8P5YYaEjzA,2,266.714294,-12.330358,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945435,-74.034157
3,7,12,8AsY4nlOJQAo8P5YYaEjzA,NoSidewalk,5.0,True,True,POINT (-74.0342025756836 40.9454460144043),8AsY4nlOJQAo8P5YYaEjzA,2,270.821442,-11.616072,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945446,-74.034203
4,7,13,8AsY4nlOJQAo8P5YYaEjzA,Crosswalk,1.0,True,True,POINT (-74.03414154052734 40.945472717285156),8AsY4nlOJQAo8P5YYaEjzA,2,284.392853,-13.758928,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945473,-74.034142
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12129,2194,13096,28tGbhCsMyFUsKRs7-jp_A,NoCurbRamp,2.0,True,True,POINT (-74.03062438964844 40.954376220703125),28tGbhCsMyFUsKRs7-jp_A,2,189.401779,-10.892858,287.290100,-2.176155,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954376,-74.030624
12130,2194,13097,28tGbhCsMyFUsKRs7-jp_A,Obstacle,4.0,NaN,True,POINT (-74.0306167602539 40.95439147949219),28tGbhCsMyFUsKRs7-jp_A,2,189.401779,-10.892858,287.290100,-2.176155,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954391,-74.030617
12131,2194,13098,5zDLTjYdw6hQhfsXppE0cA,SurfaceProblem,1.0,True,True,POINT (-74.03089141845703 40.95462417602539),5zDLTjYdw6hQhfsXppE0cA,2,77.437500,-23.928572,287.278870,-2.803400,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954624,-74.030891
12132,2202,13099,WkdNrR64GZUC1V_at1pxyA,NoCurbRamp,NaN,NaN,False,POINT (-74.02997589111328 40.96918869018555),WkdNrR64GZUC1V_at1pxyA,1,178.687500,-8.625000,281.559601,-2.859410,0cfe6980-35ac-4cdd-a2df-cbee02531406,40.969189,-74.029976


In [4]:
#read in all files in the feature folder
clustered = pd.read_csv('data/features/clustered.csv')
fp = pd.read_csv('data/features/false_positives.csv')
distance = pd.read_csv('data/features/distance.csv')
optional_input = pd.read_csv('data/features/optional_input.csv')
intersection_distance = pd.read_csv('data/features/intersection_distance.csv')

In [5]:
intersection_distance

,label_id,way_type,intersection_distance
0,9,residential,16.318492
1,10,-1,-1.000000
2,11,residential,28.909092
3,12,-1,-1.000000
4,13,-1,-1.000000
...,...,...,...
12129,13096,service,61.650246
12130,13097,-1,-1.000000
12131,13098,-1,-1.000000
12132,13099,residential,59.315314


In [6]:
#merge all into one on label_id
df = pd.merge(labels, clustered, on='label_id')
df = pd.merge(df, fp, on='label_id')
df = pd.merge(df, distance, on='label_id')
df = pd.merge(df, optional_input, on='label_id', how='inner')
df = pd.merge(df, intersection_distance, on='label_id', how='inner')

In [7]:
df

,audit_task_id,label_id,gsv_panorama_id_x,label_type,severity,correct,high_quality_user,geometry,gsv_panorama_id_y,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,lat,lng,cluster_id,count,false_positive,distance,tag_list,description,tag_count,way_type,intersection_distance
0,7,9,8AsY4nlOJQAo8P5YYaEjzA,CurbRamp,1.0,True,True,POINT (-74.03411102294922 40.945518493652344),8AsY4nlOJQAo8P5YYaEjzA,2,309.035706,-15.723214,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945518,-74.034111,442,5,0,8.785378,0.0,0.0,0.0,residential,16.318492
1,7,10,8AsY4nlOJQAo8P5YYaEjzA,NoSidewalk,5.0,True,True,POINT (-74.03414154052734 40.94554138183594),8AsY4nlOJQAo8P5YYaEjzA,2,303.232147,-13.580358,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945541,-74.034142,321,6,0,13.819353,1.0,0.0,1.0,-1,-1.000000
2,7,11,8AsY4nlOJQAo8P5YYaEjzA,CurbRamp,1.0,True,True,POINT (-74.0341567993164 40.9454345703125),8AsY4nlOJQAo8P5YYaEjzA,2,266.714294,-12.330358,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945435,-74.034157,467,5,0,16.290317,0.0,0.0,0.0,residential,28.909092
3,7,12,8AsY4nlOJQAo8P5YYaEjzA,NoSidewalk,5.0,True,True,POINT (-74.0342025756836 40.9454460144043),8AsY4nlOJQAo8P5YYaEjzA,2,270.821442,-11.616072,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945446,-74.034203,324,5,0,24.869158,1.0,0.0,1.0,-1,-1.000000
4,7,13,8AsY4nlOJQAo8P5YYaEjzA,Crosswalk,1.0,True,True,POINT (-74.03414154052734 40.945472717285156),8AsY4nlOJQAo8P5YYaEjzA,2,284.392853,-13.758928,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945473,-74.034142,159,10,0,9.900008,0.0,0.0,0.0,-1,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12129,2194,13096,28tGbhCsMyFUsKRs7-jp_A,NoCurbRamp,2.0,True,True,POINT (-74.03062438964844 40.954376220703125),28tGbhCsMyFUsKRs7-jp_A,2,189.401779,-10.892858,287.290100,-2.176155,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954376,-74.030624,7,12,1,21.503821,1.0,0.0,1.0,service,61.650246
12130,2194,13097,28tGbhCsMyFUsKRs7-jp_A,Obstacle,4.0,NaN,True,POINT (-74.0306167602539 40.95439147949219),28tGbhCsMyFUsKRs7-jp_A,2,189.401779,-10.892858,287.290100,-2.176155,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954391,-74.030617,2,11,0,20.524265,0.0,1.0,0.0,-1,-1.000000
12131,2194,13098,5zDLTjYdw6hQhfsXppE0cA,SurfaceProblem,1.0,True,True,POINT (-74.03089141845703 40.95462417602539),5zDLTjYdw6hQhfsXppE0cA,2,77.437500,-23.928572,287.278870,-2.803400,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954624,-74.030891,2,11,0,5.645213,1.0,0.0,2.0,-1,-1.000000
12132,2202,13099,WkdNrR64GZUC1V_at1pxyA,NoCurbRamp,NaN,NaN,False,POINT (-74.02997589111328 40.96918869018555),WkdNrR64GZUC1V_at1pxyA,1,178.687500,-8.625000,281.559601,-2.859410,0cfe6980-35ac-4cdd-a2df-cbee02531406,40.969189,-74.029976,5,10,1,40.275389,0.0,0.0,0.0,residential,59.315314


In [8]:
gt = pd.read_csv('data/features/oradell_ground_truth_labels.csv')

In [9]:
gt

,label_id,ground_truth
0,9,1
1,10,1
2,11,1
3,12,1
4,13,1
...,...,...
4138,11753,1
4139,12092,1
4140,12539,1
4141,12837,1


In [10]:
#merge gt with labels how left
test= pd.merge(df, gt, on='label_id', how='right')

In [11]:
#rename ground truth to verified
test.rename(columns={'ground_truth': 'verified'}, inplace=True)

In [12]:
#drop the geometry column and save as csv
test.drop(columns=['geometry']).to_csv('data/test_set_seattle.csv', index=False)

In [13]:
test.verified.value_counts()

1    3783
0     360
Name: verified, dtype: int64

In [14]:
test

,audit_task_id,label_id,gsv_panorama_id_x,label_type,severity,correct,high_quality_user,geometry,gsv_panorama_id_y,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,lat,lng,cluster_id,count,false_positive,distance,tag_list,description,tag_count,way_type,intersection_distance,verified
0,7,9,8AsY4nlOJQAo8P5YYaEjzA,CurbRamp,1.0,True,True,POINT (-74.03411102294922 40.945518493652344),8AsY4nlOJQAo8P5YYaEjzA,2,309.035706,-15.723214,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945518,-74.034111,442,5,0,8.785378,0.0,0.0,0.0,residential,16.318492,1
1,7,10,8AsY4nlOJQAo8P5YYaEjzA,NoSidewalk,5.0,True,True,POINT (-74.03414154052734 40.94554138183594),8AsY4nlOJQAo8P5YYaEjzA,2,303.232147,-13.580358,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945541,-74.034142,321,6,0,13.819353,1.0,0.0,1.0,-1,-1.000000,1
2,7,11,8AsY4nlOJQAo8P5YYaEjzA,CurbRamp,1.0,True,True,POINT (-74.0341567993164 40.9454345703125),8AsY4nlOJQAo8P5YYaEjzA,2,266.714294,-12.330358,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945435,-74.034157,467,5,0,16.290317,0.0,0.0,0.0,residential,28.909092,1
3,7,12,8AsY4nlOJQAo8P5YYaEjzA,NoSidewalk,5.0,True,True,POINT (-74.0342025756836 40.9454460144043),8AsY4nlOJQAo8P5YYaEjzA,2,270.821442,-11.616072,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945446,-74.034203,324,5,0,24.869158,1.0,0.0,1.0,-1,-1.000000,1
4,7,13,8AsY4nlOJQAo8P5YYaEjzA,Crosswalk,1.0,True,True,POINT (-74.03414154052734 40.945472717285156),8AsY4nlOJQAo8P5YYaEjzA,2,284.392853,-13.758928,108.536644,-0.496017,503d2482-420f-454e-8118-aac2d22592aa,40.945473,-74.034142,159,10,0,9.900008,0.0,0.0,0.0,-1,-1.000000,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4138,1628,11753,SNQ1WaQPKcAkvTOTAtULJA,Obstacle,2.0,True,True,POINT (-74.0452880859375 40.96165466308594),SNQ1WaQPKcAkvTOTAtULJA,2,137.830353,-11.071428,108.567726,0.086418,2d7e45f5-3656-4aae-9a09-14addfa80d22,40.961655,-74.045288,72,10,0,19.647431,1.0,0.0,1.0,-1,-1.000000,1
4139,1648,12092,F_sesLbDcaUyjG7w_nwdzg,Obstacle,4.0,True,True,POINT (-74.03170013427734 40.94500732421875),F_sesLbDcaUyjG7w_nwdzg,3,36.669643,-8.265625,19.918083,1.928528,e86fdc72-c79d-4cdc-b5b2-8ef4859141d2,40.945007,-74.031700,63,7,0,21.717830,1.0,0.0,1.0,-1,-1.000000,1
4140,1718,12539,QSi-X0zwaqMR5rhs09kYig,CurbRamp,1.0,True,True,POINT (-74.01839447021484 40.95423889160156),QSi-X0zwaqMR5rhs09kYig,1,90.812500,-14.000000,338.763458,0.073710,52b661f5-2ecd-4733-acc4-dad4b32e319e,40.954239,-74.018394,44,10,0,11.304296,0.0,0.0,0.0,residential,31.171581,1
4141,2005,12837,IZzqo_5Sjr3UZtRaV83-qA,CurbRamp,2.0,True,True,POINT (-74.0303955078125 40.953975677490234),IZzqo_5Sjr3UZtRaV83-qA,1,129.375000,-15.875000,341.502716,-1.179410,b92705c0-b6b0-42b4-8ff1-f376c0920987,40.953976,-74.030396,29,13,0,25.783018,1.0,0.0,1.0,residential,34.700662,1


In [15]:
#if the label_id in gt take it out of df
training = df[~df.label_id.isin(test.label_id)]

In [16]:
training

,audit_task_id,label_id,gsv_panorama_id_x,label_type,severity,correct,high_quality_user,geometry,gsv_panorama_id_y,zoom,heading,pitch,photographer_heading,photographer_pitch,user_id,lat,lng,cluster_id,count,false_positive,distance,tag_list,description,tag_count,way_type,intersection_distance
48,8,59,7l-j4D7uhfKRQPpYXIU59w,Obstacle,2.0,True,True,POINT (-74.03707122802734 40.94615936279297),7l-j4D7uhfKRQPpYXIU59w,3,225.573654,-11.361608,20.765854,0.262733,503d2482-420f-454e-8118-aac2d22592aa,40.946159,-74.037071,115,8,0,10.614774,1.0,0.0,1.0,-1,-1.000000
117,18,137,adXdsY189n44f0_ae5Hptg,NoCurbRamp,5.0,False,True,POINT (-74.03673553466797 40.96704864501953),adXdsY189n44f0_ae5Hptg,1,278.325348,-27.120001,291.724182,3.633377,e86fdc72-c79d-4cdc-b5b2-8ef4859141d2,40.967049,-74.036736,403,6,0,1.638389,1.0,0.0,1.0,primary,35.635586
119,18,139,adXdsY189n44f0_ae5Hptg,SurfaceProblem,1.0,True,True,POINT (-74.03650665283203 40.96693420410156),adXdsY189n44f0_ae5Hptg,2,158.084290,-17.280714,291.724182,3.633377,e86fdc72-c79d-4cdc-b5b2-8ef4859141d2,40.966934,-74.036507,162,10,0,21.534606,1.0,0.0,2.0,-1,-1.000000
122,18,142,adXdsY189n44f0_ae5Hptg,SurfaceProblem,1.0,True,True,POINT (-74.03662109375 40.96712112426758),adXdsY189n44f0_ae5Hptg,2,324.155701,-22.816429,291.724182,3.633377,e86fdc72-c79d-4cdc-b5b2-8ef4859141d2,40.967121,-74.036621,2233,4,0,34.130406,1.0,0.0,2.0,-1,-1.000000
130,19,167,QnbhyLuq-5js6HTPvM7sKw,NoSidewalk,NaN,True,True,POINT (-74.03386688232422 40.96540451049805),QnbhyLuq-5js6HTPvM7sKw,1,150.875000,-21.783482,5.927903,-0.361237,e86fdc72-c79d-4cdc-b5b2-8ef4859141d2,40.965405,-74.033867,2886,2,0,6.722622,0.0,0.0,0.0,-1,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12129,2194,13096,28tGbhCsMyFUsKRs7-jp_A,NoCurbRamp,2.0,True,True,POINT (-74.03062438964844 40.954376220703125),28tGbhCsMyFUsKRs7-jp_A,2,189.401779,-10.892858,287.290100,-2.176155,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954376,-74.030624,7,12,1,21.503821,1.0,0.0,1.0,service,61.650246
12130,2194,13097,28tGbhCsMyFUsKRs7-jp_A,Obstacle,4.0,NaN,True,POINT (-74.0306167602539 40.95439147949219),28tGbhCsMyFUsKRs7-jp_A,2,189.401779,-10.892858,287.290100,-2.176155,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954391,-74.030617,2,11,0,20.524265,0.0,1.0,0.0,-1,-1.000000
12131,2194,13098,5zDLTjYdw6hQhfsXppE0cA,SurfaceProblem,1.0,True,True,POINT (-74.03089141845703 40.95462417602539),5zDLTjYdw6hQhfsXppE0cA,2,77.437500,-23.928572,287.278870,-2.803400,528eb1d6-e20e-4fb9-81c9-0c06b9e2852f,40.954624,-74.030891,2,11,0,5.645213,1.0,0.0,2.0,-1,-1.000000
12132,2202,13099,WkdNrR64GZUC1V_at1pxyA,NoCurbRamp,NaN,NaN,False,POINT (-74.02997589111328 40.96918869018555),WkdNrR64GZUC1V_at1pxyA,1,178.687500,-8.625000,281.559601,-2.859410,0cfe6980-35ac-4cdd-a2df-cbee02531406,40.969189,-74.029976,5,10,1,40.275389,0.0,0.0,0.0,residential,59.315314


In [17]:
training.to_csv('data/processed-labels/training_set_oradell.csv', index=False)
test.to_csv('data/processed-labels/test_set_oradell.csv', index=False)
df.to_csv('data/processed-labels/labels_full_features_oradell.csv', index=False)